In [20]:
from pyspark.sql import SparkSession

In [21]:
spark =  SparkSession.builder.appName('mylogreg12').getOrCreate()

In [22]:
df = spark.read.csv('titanic.csv', inferSchema=True, header= True)

In [24]:
df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [25]:
df.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [27]:
my_cols = df.select(['Survived',
 'Pclass','Sex',
 'Age',
 'SibSp',
 'Parch','Embarked','Fare'])

In [28]:
my_final_Data = my_cols.na.drop()

In [29]:
from pyspark.ml.feature import (VectorAssembler, VectorIndexer,
                                   OneHotEncoder,StringIndexer)

In [30]:
gender_indexer = StringIndexer(inputCol='Sex',outputCol='SexIndex')
embark_indexer = StringIndexer(inputCol='Embarked',outputCol='EmbarkIndex')

In [31]:
gender_encoder = OneHotEncoder(inputCol='SexIndex',outputCol='SexVec')

In [32]:
embark_encoder = OneHotEncoder(inputCol='EmbarkIndex',outputCol='EmbarkVec')

In [33]:
assembler = VectorAssembler(inputCols=['Pclass','SexVec','EmbarkVec','Age','SibSp','Parch','Fare'],outputCol='features')

In [34]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

In [35]:
log_reg_titanic = LogisticRegression(featuresCol='features',labelCol='Survived')

In [36]:
pipeline = Pipeline(stages = [gender_indexer,embark_indexer,embark_encoder,gender_encoder,assembler,log_reg_titanic])

In [37]:
train_data, test_data = my_final_Data.randomSplit([0.7,0.3])

In [38]:
fit_model = pipeline.fit(train_data)

In [39]:
results = fit_model.transform(test_data)

In [40]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [42]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol = 'Survived')

In [43]:
results.select('Survived','prediction').show()

+--------+----------+
|Survived|prediction|
+--------+----------+
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       0.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
+--------+----------+
only showing top 20 rows



In [44]:
AUC =my_eval.evaluate(results)

In [45]:
AUC

0.7383150039277299